In [1]:
import sys
import os

if sys.version_info[0] < 3:
  print('[ERROR] You need to run this with Python 3.')
  raise AssertionError

In [2]:
import numpy as np

import functools
import itertools
import toolz

from emtf_algos import *
from emtf_logger import get_logger
from emtf_colormap import get_colormap

In [3]:
# Set random seed
np.random.seed(2027)

import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import layers as k_layers
from tensorflow.keras import backend as k_backend
import matplotlib as mpl
import matplotlib.pyplot as plt

# Set random seed
tf.random.set_seed(2027)

#import numba
#from numba import njit, vectorize
import dask
import dask.array as da

logger = get_logger()
logger.info('Using cmssw      : {0}'.format(os.environ['CMSSW_VERSION'] if 'CMSSW_VERSION' in os.environ else 'n/a'))
logger.info('Using python     : {0}'.format(sys.version.replace('\n', '')))
logger.info('Using numpy      : {0}'.format(np.__version__))
logger.info('Using tensorflow : {0}'.format(tf.__version__))
logger.info('Using keras      : {0}'.format(keras.__version__))
logger.info('.. list devices  : {0}'.format(tf.config.list_physical_devices()))
logger.info('Using matplotlib : {0}'.format(mpl.__version__))
#logger.info('Using numba      : {0}'.format(numba.__version__))
logger.info('Using dask       : {0}'.format(dask.__version__))

assert k_backend.backend() == 'tensorflow'
assert k_backend.image_data_format() == 'channels_last'

%matplotlib inline

[INFO    ] Using cmssw      : CMSSW_10_6_3
[INFO    ] Using python     : 3.6.10 |Anaconda, Inc.| (default, Mar 25 2020, 23:51:54) [GCC 7.3.0]
[INFO    ] Using numpy      : 1.19.2
[INFO    ] Using tensorflow : 2.4.0
[INFO    ] Using keras      : 2.4.0
[INFO    ] .. list devices  : [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]
[INFO    ] Using matplotlib : 3.3.2
[INFO    ] Using dask       : 2020.12.0


In [4]:
# Settings

# zone: (0,1,2) -> eta=(1.98..2.5, 1.55..1.98, 1.2..1.55)
zone = 0
#zone = 1
#zone = 2

# timezone: (0,1,2) -> BX=(0,-1,-2)
timezone = 0

# masked array filling value
ma_fill_value = 999999

#maxevents = 10
maxevents = -1

#workers = 1
workers = 8

# Input files
patterns_fname = 'patterns_zone%i.npz' % zone
zone_images_fname = 'zone_images_zone%i.h5' % zone

# Scheduler
dask.config.set(scheduler='threads', num_workers=workers)

# Styling
plt.style.use('tdrstyle.mplstyle')
cm = get_colormap()

logger.info('Processing zone {0} timezone {1}'.format(zone, timezone))
logger.info('.. maxevents        : {0}'.format(maxevents))
logger.info('.. workers          : {0}'.format(workers))

[INFO    ] Processing zone 0 timezone 0
[INFO    ] .. maxevents        : -1
[INFO    ] .. workers          : 8


### Load data

In [5]:
import h5py
file_handles = []

def load_patterns():
  patterns = []
  boxes_act = []
  hitmap_quality_ranks = []
  for i in range(num_emtf_zones):
    fname = patterns_fname.replace('zone%i' % zone, 'zone%i' % i)  # modify filename
    logger.info('Loading from {0}'.format(fname))
    with np.load(fname) as loaded:
      patterns.append(loaded['patterns'])
      boxes_act.append(loaded['boxes_act'])
      hitmap_quality_ranks.append(loaded['hitmap_quality_ranks'])
  patterns = np.asarray(patterns)
  boxes_act = np.asarray(boxes_act)
  hitmap_quality_ranks = np.asarray(hitmap_quality_ranks)
  logger.info('patterns: {0} boxes_act: {1} hitmap_quality_ranks: {2}'.format(patterns.shape, boxes_act.shape, hitmap_quality_ranks.shape))
  return patterns, boxes_act, hitmap_quality_ranks

def load_zone_sparse_images(fname):
  logger.info('Loading from {0}'.format(fname))
  loaded = h5py.File(fname, 'r')
  file_handles.append(loaded)
  zone_box_anchors = loaded['zone_box_anchors']
  zone_sparse_images = SparseTensorValue(indices=loaded['zone_sparse_images_indices'],
                                         values=loaded['zone_sparse_images_values'],
                                         dense_shape=loaded['zone_sparse_images_dense_shape'])
  logger.info('zone_box_anchors: {0} zone_sparse_images: {1}'.format(zone_box_anchors.shape, zone_sparse_images.dense_shape))
  return zone_box_anchors, zone_sparse_images

def load_zone_hits(fname):
  logger.info('Loading from {0}'.format(fname))
  loaded = h5py.File(fname, 'r')
  file_handles.append(loaded)
  zone_part = loaded['zone_part']
  zone_hits = RaggedTensorValue(values=loaded['zone_hits_values'],
                                row_splits=loaded['zone_hits_row_splits'])
  zone_simhits = RaggedTensorValue(values=loaded['zone_simhits_values'],
                                   row_splits=loaded['zone_simhits_row_splits'])
  logger.info('zone_part: {0} zone_hits: {1} zone_simhits: {2}'.format(zone_part.shape, zone_hits.shape, zone_simhits.shape))
  return zone_part, zone_hits, zone_simhits

def load_zone_hits_lazy(fname):
  logger.info('Loading from {0}'.format(fname))
  loaded = h5py.File(fname, 'r')
  file_handles.append(loaded)
  zone_part = da.from_array(loaded['zone_part'])
  zone_hits_values = da.from_array(loaded['zone_hits_values'])
  zone_hits_row_splits = da.from_array(loaded['zone_hits_row_splits'])
  zone_hits_shape = (zone_hits_row_splits.shape[0] - 1,) + (None,) + zone_hits_values.shape[1:]
  zone_simhits_values = da.from_array(loaded['zone_simhits_values'])
  zone_simhits_row_splits = da.from_array(loaded['zone_simhits_row_splits'])
  zone_simhits_shape = (zone_simhits_row_splits.shape[0] - 1,) + (None,) + zone_simhits_values.shape[1:]
  logger.info('zone_part: {0} zone_hits: {1} zone_simhits: {2}'.format(zone_part.shape, zone_hits_shape, zone_simhits_shape))
  return zone_part, (zone_hits_values, zone_hits_row_splits), (zone_simhits_values, zone_simhits_row_splits)

In [6]:
# Load patterns
patterns, boxes_act, hitmap_quality_ranks = load_patterns()

# Create patterns_reshaped
patterns_reshaped = []
for i in range(num_emtf_zones):
  p = patterns[i, 3, [3, 2, 4, 1, 5, 0, 6]]  # order by straightness, only prompt patterns
  patterns_reshaped.append(p)
patterns_reshaped = np.asarray(patterns_reshaped)
logger.info('patterns_reshaped: {0}'.format(patterns_reshaped.shape))

# Create boxes_act_reshaped
boxes_act_reshaped = []
for i in range(num_emtf_zones):
  b = boxes_act[i, 3, [3, 2, 4, 1, 5, 0, 6]]  # order by straightness, only prompt patterns
  b = np.transpose(b, [3, 2, 1, 0])  # kernel shape is HWCD
  boxes_act_reshaped.append(b)
boxes_act_reshaped = np.asarray(boxes_act_reshaped)
logger.info('boxes_act_reshaped: {0}'.format(boxes_act_reshaped.shape))

# Create boxes_qual_reshaped
boxes_qual_reshaped = hitmap_quality_ranks // 4  # from 8-bit to 6-bit
assert boxes_qual_reshaped.max() == 63
logger.info('boxes_qual_reshaped: {0}'.format(boxes_qual_reshaped.shape))

[INFO    ] Loading from patterns_zone0.npz
[INFO    ] Loading from patterns_zone1.npz
[INFO    ] Loading from patterns_zone2.npz
[INFO    ] patterns: (3, 7, 7, 8, 3) boxes_act: (3, 7, 7, 8, 111, 1) hitmap_quality_ranks: (3, 256)
[INFO    ] patterns_reshaped: (3, 7, 8, 3)
[INFO    ] boxes_act_reshaped: (3, 1, 111, 8, 7)
[INFO    ] boxes_qual_reshaped: (3, 256)


In [7]:
# Load zone_hits (lazily)
zone_part_l, zone_hits_l, zone_simhits_l = load_zone_hits_lazy(zone_images_fname)

[INFO    ] Loading from zone_images_zone0.h5
[INFO    ] zone_part: (652591, 9) zone_hits: (652591, None, 18) zone_simhits: (652591, None, 18)


### Create inputs

In [8]:
# Input data columns
hits_metadata = ['emtf_site', 'emtf_host', 'emtf_chamber',
                 'emtf_segment', 'zones', 'timezones',
                 'emtf_phi', 'emtf_bend', 'emtf_theta',
                 'emtf_theta_alt', 'emtf_qual', 'emtf_qual_alt',
                 'emtf_time', 'strip', 'wire',
                 'fr', 'detlayer', 'bx']
hits_metadata = dict(zip(hits_metadata, range(len(hits_metadata))))
#print(hits_metadata)

# Image format
num_channels = 1
num_cols = 288  # 80 degrees
num_rows = 8
image_format = (num_rows, num_cols, num_channels)

# Array indices
ind_emtf_chamber = hits_metadata['emtf_chamber']
ind_emtf_segment = hits_metadata['emtf_segment']

ind_emtf_phi = hits_metadata['emtf_phi']
ind_emtf_bend = hits_metadata['emtf_bend']
ind_emtf_theta1 = hits_metadata['emtf_theta']
ind_emtf_theta2 = hits_metadata['emtf_theta_alt']
ind_emtf_qual1 = hits_metadata['emtf_qual']
ind_emtf_qual2 = hits_metadata['emtf_qual_alt']
ind_emtf_time = hits_metadata['emtf_time']
ind_zones = hits_metadata['zones']
ind_tzones = hits_metadata['timezones']
ind_fr = hits_metadata['fr']
ind_dl = hits_metadata['detlayer']
ind_bx = hits_metadata['bx']
ind_valid = hits_metadata['bx']  # CUIDADO: use 'bx' for the moment

new_hits_metadata = ['emtf_phi', 'emtf_bend', 'emtf_theta1',
                     'emtf_theta2', 'emtf_qual1', 'emtf_qual2',
                     'emtf_time', 'zones', 'tzones',
                     'fr', 'dl', 'bx',
                     'valid']
new_hits_metadata = dict(zip(new_hits_metadata, range(len(new_hits_metadata))))
#print(new_hits_metadata)

In [9]:
class RaggedTensorLazyValue(object):
  def __init__(self, values, row_splits):
    _row_splits = np.array(row_splits)  # lazy no more
    _values = np.array(values[:_row_splits[-1]])  # lazy no more
    self.values = _values
    self.row_splits = _row_splits
    self.shape = (self.row_splits.shape[0] - 1,) + (None,) + self.values.shape[1:]

  def __getitem__(self, row_key):
    starts = self.row_splits[:-1]
    limits = self.row_splits[1:]
    row = self.values[starts[row_key]:limits[row_key]]
    return row

@dask.delayed
def build_inputs(ievt):
  zone_hits_ievt = zone_hits[ievt]
  zone_hits_columns = [ind_emtf_chamber, ind_emtf_segment]
  indices = zone_hits_ievt[:, zone_hits_columns]
  zone_hits_columns = [ind_emtf_phi, ind_emtf_bend, ind_emtf_theta1, ind_emtf_theta2,
                       ind_emtf_qual1, ind_emtf_qual2, ind_emtf_time, ind_zones,
                       ind_tzones, ind_fr, ind_dl, ind_bx,
                       ind_valid]
  values = zone_hits_ievt[:, zone_hits_columns]
  values[:, -1] = 1  # CUIDADO: set 'valid' to 1

  # Apply truncation
  valid = indices[:, 1] < num_emtf_segments
  indices = indices[valid]
  values = values[valid]

  # Sparse -> Dense
  dense_shape = np.array([num_emtf_chambers, num_emtf_segments, num_emtf_variables], dtype=np.int32)
  sparse = SparseTensorValue(indices=indices, values=values, dense_shape=dense_shape)
  dense = sparse_to_dense(sparse)

  # Result
  result = (dense, np.concatenate((indices, values), axis=-1))
  return result

def build_inputs_batch(batch_ids):
  results = map(build_inputs, batch_ids)
  return results

def build_inputs_all():
  # Split into batches
  batches = np.array_split(np.arange(zone_hits.shape[0]), workers)
  results, = dask.compute(map(build_inputs_batch, batches))  # delayed no more

  # Chain and zip results
  results = list(itertools.chain.from_iterable(results))
  inputs, sparse_inputs = zip(*results)
  return (np.asarray(inputs), sparse_inputs)

In [10]:
# Build inputs

if maxevents == -1:
  zone_hits = RaggedTensorLazyValue(values=zone_hits_l[0], row_splits=zone_hits_l[1])
else:
  zone_hits = RaggedTensorLazyValue(values=zone_hits_l[0], row_splits=zone_hits_l[1][:(maxevents + 1)])

zone_hits_l = None  # release from memory
for file_handle in file_handles:  # close files
  file_handle.close()

inputs, sparse_inputs = build_inputs_all()
sparse_inputs_shape = (len(sparse_inputs),) + (None,) + (sparse_inputs[0].shape[-1],)
logger.info('inputs: {0} sparse_inputs: {1}'.format(inputs.shape, sparse_inputs_shape))

[INFO    ] inputs: (652591, 115, 2, 13) sparse_inputs: (652591, None, 15)


In [11]:
# Debug
def tf_constant_value(tensor):
  if tf.is_tensor(tensor):
    try:
      return tensor.numpy()
    except:
      return tensor
  else:
    return tensor

my_array2string = functools.partial(np.array2string, separator=', ', formatter={'int':lambda x: '% 4i' % x}, max_line_width=100, threshold=1000)

print(my_array2string(sparse_inputs[0]))
print(my_array2string(sparse_inputs[2]))
print(my_array2string(sparse_inputs[5]))

[[   2,    0,  2548,    5,   18,   17,    6,    6,    0,    4,    4,    0,    0,    0,    1],
 [  19,    0,  2684,    2,   16,   16,    6,    6,    0,    4,    4,    1,    0,    0,    1],
 [  28,    0,  2819,   15,   17,   16,    5,    5,    0,    4,    4,    0,    0,    0,    1],
 [  28,    1,  2728,    0,   16,   17,    5,    5,    0,    4,    4,    0,    0,    0,    1],
 [  37,    0,  2736,    0,   16,   16,    5,    5,    0,    4,    4,    0,    0,    0,    1],
 [  55,    0,  2505,    0,   18,   18,    2,    2,    0,    4,    4,    1,    0,    0,    1],
 [  73,    0,  2675,    0,   19,   19,    2,    2,    0,    4,    4,    1,    0,    0,    1],
 [  82,    0,  2888,    0,   17,   17,    1,    1,    1,    4,    4,    0,    0,    0,    1],
 [  82,    1,  2714,    0,   17,   17,    5,    5,    1,    4,    4,    0,    0,    0,    1],
 [  91,    0,  2737,    0,   17,   17,    1,    1,    0,    4,    4,    0,    0,    0,    1],
 [ 109,    0,  2479,   15,   17,   17,    6,    6,    0,    

### Create model

In [12]:
def prepare_constants():
  # Utility functions & LUTs
  inverse_fn = lambda F, y: [[i for (i, y_i) in enumerate(F) if y_i == y_j] for y_j in y]
  to_array = lambda x: np.asarray([np.asarray(x_i) for x_i in x])
  to_list = lambda x: [x_i.tolist() for x_i in x]
  flatten = lambda x: np.asarray([x_i_i for x_i in x for x_i_i in x_i])

  def to_array(x):  # improved version
    is_ragged = len(set([len(x_i) for x_i in x])) > 1
    if is_ragged:
      return np.asarray([np.asarray(x_i) for x_i in x], dtype=np.object)
    else:
      return np.asarray([x_i for x_i in x])

  class Constants:
    pass

  cc = Constants()

  cc.site_to_img_row_luts = site_to_img_row_luts

  host_to_chamber_lut = to_array(inverse_fn(chamber_to_host_lut, np.arange(num_emtf_hosts)))
  #host_to_chamber_lut_flat = flatten(host_to_chamber_lut)

  site_to_host_lut = to_array(inverse_fn(host_to_site_lut, np.arange(num_emtf_sites)))
  site_to_chamber_lut = to_array([
      [c for host in hosts for c in host_to_chamber_lut[host]] \
      for hosts in site_to_host_lut
  ])
  site_numbers = to_array([
      np.repeat(i, len(x)) for i, x in enumerate(site_to_chamber_lut)
  ])
  cc.site_to_chamber_lut = site_to_chamber_lut
  cc.site_numbers = site_numbers

  cc.img_row_to_chamber_lut = np.empty(num_emtf_zones, dtype=np.object)
  cc.img_row_numbers = np.empty(num_emtf_zones, dtype=np.object)

  for i in range(num_emtf_zones):
    host_to_img_row_lut = find_emtf_img_row_lut()[:, i]
    img_row_to_host_lut = to_array(inverse_fn(host_to_img_row_lut, np.arange(num_rows)))
    img_row_to_chamber_lut = to_array([
        [c for host in hosts for c in host_to_chamber_lut[host]] \
        for hosts in img_row_to_host_lut
    ])
    img_row_numbers = to_array([
        np.repeat(i, len(x)) for i, x in enumerate(img_row_to_chamber_lut)
    ])
    cc.img_row_to_chamber_lut[i] = img_row_to_chamber_lut
    cc.img_row_numbers[i] = img_row_numbers
  return cc

cc = prepare_constants()

In [13]:
def build_zone_image(x_ievt, zone, timezone, image_format):
  def get_boolean_mask(zone, row):
    indices = cc.img_row_to_chamber_lut[zone][row]
    boolean_mask = np.zeros(num_emtf_chambers, dtype=np.bool)
    boolean_mask[indices] = 1
    return boolean_mask

  new_ind_emtf_phi = new_hits_metadata['emtf_phi']
  new_ind_zones = new_hits_metadata['zones']
  new_ind_tzones = new_hits_metadata['tzones']
  new_ind_valid = new_hits_metadata['valid']

  x_emtf_phi = x_ievt[..., new_ind_emtf_phi]
  x_zones = x_ievt[..., new_ind_zones]
  x_tzones = x_ievt[..., new_ind_tzones]
  x_valid = x_ievt[..., new_ind_valid]

  # Valid for this zone
  _valid = (x_valid == 1) & \
           (x_zones & (1<<((num_emtf_zones - 1) - zone))).astype(np.bool) & \
           (x_tzones & (1<<((num_emtf_timezones - 1) - timezone))).astype(np.bool)

  # Prepare zone image
  img = np.zeros(image_format, dtype=np.bool)

  # Loop over rows
  for row in range(image_format[0]):
    boolean_mask = get_boolean_mask(zone, row)
    valid = _valid[boolean_mask]
    col = find_emtf_img_col(x_emtf_phi[boolean_mask][valid])
    row = (col * 0) + row
    channel = (col * 0)
    img[(row, col, channel)] = 1  # fill
  return img

In [14]:
def build_track_cand(x_ievt, x, idx_h, idx_w, idx_z, num_features):
  def get_phi_patt(zone, patt, row, col, _which):
    # In the following, (0, 0, 4) is zone 0 patt 'straightest' row 'ME2/1'
    if _which == 'start':
      translation = patterns_reshaped[zone, patt, row, 0] - patterns_reshaped[0, 0, 4, 1]
    elif _which == 'mid':
      translation = patterns_reshaped[zone, patt, row, 1] - patterns_reshaped[0, 0, 4, 1]
    elif _which == 'stop':
      translation = patterns_reshaped[zone, patt, row, 2] - patterns_reshaped[0, 0, 4, 1]
    else:
      raise ValueError('Invalid: %s' % _which)
    col = col + translation
    phi_patt = find_emtf_img_col_inverse(col)
    return phi_patt

  def get_img_row(zone, site):
    return cc.site_to_img_row_luts[zone][site].item()

  def get_boolean_mask(zone, site):
    indices = cc.site_to_chamber_lut[site]
    boolean_mask = np.zeros(num_emtf_chambers, dtype=np.bool)
    boolean_mask[indices] = 1
    return boolean_mask

  new_ind_emtf_phi = new_hits_metadata['emtf_phi']
  new_ind_emtf_bend = new_hits_metadata['emtf_bend']
  new_ind_emtf_theta1 = new_hits_metadata['emtf_theta1']
  new_ind_emtf_theta2 = new_hits_metadata['emtf_theta2']
  new_ind_emtf_qual1 = new_hits_metadata['emtf_qual1']
  new_ind_emtf_qual2 = new_hits_metadata['emtf_qual2']
  new_ind_emtf_time = new_hits_metadata['emtf_time']
  new_ind_zones = new_hits_metadata['zones']
  new_ind_tzones = new_hits_metadata['tzones']
  new_ind_valid = new_hits_metadata['valid']

  x_emtf_phi = x_ievt[..., new_ind_emtf_phi]
  x_emtf_bend = x_ievt[..., new_ind_emtf_bend]
  x_emtf_theta1 = x_ievt[..., new_ind_emtf_theta1]
  x_emtf_theta2 = x_ievt[..., new_ind_emtf_theta2]
  x_emtf_qual1 = x_ievt[..., new_ind_emtf_qual1]
  x_emtf_qual2 = x_ievt[..., new_ind_emtf_qual2]
  x_emtf_time = x_ievt[..., new_ind_emtf_time]
  x_zones = x_ievt[..., new_ind_zones]
  x_tzones = x_ievt[..., new_ind_tzones]
  x_valid = x_ievt[..., new_ind_valid]

  x_seg = np.arange(num_emtf_chambers * num_emtf_segments).reshape((num_emtf_chambers, num_emtf_segments))

  invalid_marker_ph_seg = (num_emtf_chambers * num_emtf_segments)

  (zone, patt, col, qual, bx) = (idx_z.item(), idx_h.item(), idx_w.item(), x.item(), 0)

  # Prepare track_cand, track_cand_seg
  track_cand = np.zeros(num_features, dtype=np.int32)
  track_cand_seg = np.zeros(num_emtf_sites, dtype=np.int32) + invalid_marker_ph_seg

  num_feature_axes = 7  # (emtf_phi, emtf_bend, emtf_theta1, emtf_theta2, emtf_qual1, emtf_qual2, emtf_time)
  feature_values = np.zeros(num_feature_axes * num_emtf_sites, dtype=x_ievt.dtype)

  # Valid for this zone
  _valid = (x_valid == 1) & \
           (x_zones & (1<<((num_emtf_zones - 1) - zone))).astype(np.bool) & \
           (x_tzones & (1<<((num_emtf_timezones - 1) - timezone))).astype(np.bool)

  # Loop over sites
  for site in range(num_emtf_sites):
    row = get_img_row(zone, site)
    phi_patt_start = get_phi_patt(zone, patt, row, col, 'start')
    phi_patt_mid = get_phi_patt(zone, patt, row, col, 'mid')
    phi_patt_stop = get_phi_patt(zone, patt, row, col, 'stop')

    _valid_phi = (find_emtf_img_col(phi_patt_start) <= find_emtf_img_col(x_emtf_phi)) & \
                 (find_emtf_img_col(x_emtf_phi) <= find_emtf_img_col(phi_patt_stop))

    boolean_mask = get_boolean_mask(zone, site)
    valid = (_valid & _valid_phi)[boolean_mask]
    phi_values = x_emtf_phi[boolean_mask][valid]

    if len(phi_values):
      # Select min dphi
      dphi = np.abs(phi_values - phi_patt_mid)
      idx = np.argmin(dphi)

      # Set segment indices
      track_cand_seg[site] = x_seg[boolean_mask][valid][idx]

      # Set feature_values
      feature_values[np.arange(num_feature_axes) * num_emtf_sites + site] = [
          x_emtf_phi[boolean_mask][valid][idx],
          x_emtf_bend[boolean_mask][valid][idx],
          x_emtf_theta1[boolean_mask][valid][idx],
          x_emtf_theta2[boolean_mask][valid][idx],
          x_emtf_qual1[boolean_mask][valid][idx],
          x_emtf_qual2[boolean_mask][valid][idx],
          x_emtf_time[boolean_mask][valid][idx],
      ]
      # End if len(phi_values)
    # End loop over site

  # Find phi_median and theta_median
  get_ind_emtf_phi_at_site = lambda site: (new_ind_emtf_phi * num_emtf_sites) + site
  get_ind_emtf_theta1_at_site = lambda site: (new_ind_emtf_theta1 * num_emtf_sites) + site
  get_ind_emtf_theta2_at_site = lambda site: (new_ind_emtf_theta2 * num_emtf_sites) + site

  theta_values = feature_values[[
      get_ind_emtf_theta1_at_site(2),
      get_ind_emtf_theta1_at_site(3),
      get_ind_emtf_theta1_at_site(4),
      get_ind_emtf_theta2_at_site(2),
      get_ind_emtf_theta2_at_site(3),
      get_ind_emtf_theta2_at_site(4),
      get_ind_emtf_theta1_at_site(6),
      get_ind_emtf_theta1_at_site(7),
      get_ind_emtf_theta1_at_site(8),
  ]]
  theta_values_s1 = feature_values[[
      get_ind_emtf_theta1_at_site(1),
      get_ind_emtf_theta1_at_site(0),
      get_ind_emtf_theta1_at_site(11),
      get_ind_emtf_theta2_at_site(1),
      get_ind_emtf_theta2_at_site(0),
      get_ind_emtf_theta2_at_site(11),
      get_ind_emtf_theta1_at_site(5),
      get_ind_emtf_theta1_at_site(9),
      get_ind_emtf_theta1_at_site(11),
  ]]

  if theta_values[6] == 0:
    theta_values[6] = feature_values[get_ind_emtf_theta1_at_site(10)]
  theta_values_s1[2] = 0
  theta_values_s1[5] = 0

  def find_theta_median():
    def find_median_of_three(x):
      x = np.sort(x)
      return pick_the_median(x[x > 0]) if np.any(x > 0) else pick_the_first(x)
    #
    def find_median_of_nine(x):
      fn = find_median_of_three
      return fn((fn(x[0:3]), fn(x[3:6]), fn(x[6:9])))
    #
    if np.any(theta_values > 0):
      return find_median_of_nine(theta_values)
    else:
      return find_median_of_nine(theta_values_s1)

  phi_median = find_emtf_img_col_inverse(col)
  theta_median = find_theta_median()

  # Require theta window, find best theta values
  valid_track = False
  for site in range(num_emtf_sites):
    theta1 = feature_values[get_ind_emtf_theta1_at_site(site)]
    theta2 = feature_values[get_ind_emtf_theta2_at_site(site)]
    dtheta1 = np.abs(theta1 - theta_median)
    dtheta2 = np.abs(theta2 - theta_median)
    th_window = 8
    valid_site = (theta1 != 0 and theta2 != 0) and (dtheta1 < th_window or dtheta2 < th_window)
    valid_track = valid_track or valid_site
    if dtheta2 < dtheta1:
      feature_values[get_ind_emtf_theta1_at_site(site)] = theta2

    feature_values[get_ind_emtf_phi_at_site(site)] -= phi_median
    feature_values[get_ind_emtf_theta1_at_site(site)] -= theta_median
    if not valid_site:
      feature_values[np.arange(num_feature_axes) * num_emtf_sites + site] = ma_fill_value

  # Extract features
  feature_values_indices = [
     0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11,  # emtf_phi
    24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,  # emtf_theta
    12, 13, 14, 15, 16, 23,                          # emtf_bend
    48, 49, 50, 51, 52, 59,                          # emtf_qual
  ]
  phi_median_1 = (phi_median - find_emtf_img_col_inverse(num_cols // 2))
  additional_features = [phi_median_1, theta_median, qual, bx]
  additional_features = np.array(additional_features, dtype=x_ievt.dtype)
  if not valid_track:
    additional_features[:] = 0

  track_cand[:len(feature_values_indices)] = feature_values[feature_values_indices]
  track_cand[len(feature_values_indices):] = additional_features
  return (track_cand, track_cand_seg)

In [15]:
def remove_track_dupes(track_cands, track_cands_seg, num_features, num_tracks):
  def find_seg_rm(x):
    mask = (x != invalid_marker_ph_seg)
    return pick_the_first(x[mask]) if np.any(mask) else pick_the_first(x)

  invalid_marker_ph_seg = (num_emtf_chambers * num_emtf_segments)

  track_cands_seg_indices = np.array([[0, 9, 1, 5], [2, 10, 6], [3, 7], [4, 8], [11]], dtype=np.object)
  assert(len(track_cands_seg_indices) == num_emtf_sites_rm)

  # Prepare track_cands_rm, track_cands_seg_rm
  track_cands_rm = np.zeros(track_cands.shape, dtype=track_cands.dtype)
  track_cands_seg_rm = np.zeros(track_cands_seg.shape, dtype=track_cands_seg.dtype) + invalid_marker_ph_seg

  # Loop over tracks
  for itrk in range(num_tracks):
    for site in range(num_emtf_sites_rm):
      track_cands_seg_rm[itrk, site] = find_seg_rm(track_cands_seg[itrk, track_cands_seg_indices[site]])

  # Mark for removal
  removal = np.zeros(num_tracks, dtype=np.bool)
  for i in range(num_tracks - 1):
    for j in range(i + 1, num_tracks):
      removal[j] |= np.any(
          (track_cands_seg_rm[i] != invalid_marker_ph_seg) & \
          (track_cands_seg_rm[j] != invalid_marker_ph_seg) & \
          (track_cands_seg_rm[i] == track_cands_seg_rm[j])
      )

  # Gather survivors
  i = 0
  for j in range(num_tracks):
    if not removal[j]:
      track_cands_rm[i] = track_cands[j]
      i += 1
  if i != num_tracks:
    track_cands_rm[i:] = ma_fill_value
  return track_cands_rm

In [16]:
# Creating custom layers
# See: https://www.tensorflow.org/tutorials/customization/custom_layers

class Zoning(k_layers.Layer):
  def __init__(self, zone, timezone, image_format=image_format, **kwargs):
    super(Zoning, self).__init__(**kwargs)
    self.zone = zone
    self.timezone = timezone
    self.image_format = image_format

    # Set up py_func
    kwargs = dict(zone=self.zone, timezone=self.timezone, image_format=self.image_format)
    _build_zone_image = functools.partial(build_zone_image, **kwargs)
    #self.py_func = lambda x: tf.py_function(_build_zone_image, [x], tf.bool)
    self.py_func = lambda x: tf.numpy_function(_build_zone_image, [x], tf.bool)
    self.py_func_1 = lambda x: tf.map_fn(self.py_func, x, fn_output_signature=tf.bool)

  def call(self, inputs):
    x_ievt = inputs

    # Call py_func
    x = self.py_func_1(x_ievt)
    output_shape = (None,) + self.image_format
    x.set_shape(output_shape)
    return x

class Pooling(k_layers.Layer):
  def __init__(self, zone, image_format=image_format, num_patterns=num_emtf_patterns, **kwargs):
    super(Pooling, self).__init__(**kwargs)
    self.zone = zone
    self.image_format = image_format
    self.num_patterns = num_patterns

    # SeparableConv2D but only the depthwise conv (i.e. without the pointwise conv)
    # See: https://www.tensorflow.org/api_docs/python/tf/keras/layers/DepthwiseConv2D
    # See: https://www.tensorflow.org/api_docs/python/tf/keras/layers/SeparableConv2D
    from k_layers_separable_conv2d import SeparableConv2D as MySeparableConv2D
    w_init = keras.initializers.Constant(boxes_act_reshaped[self.zone])
    conv2d_kwargs = dict(filters=1, kernel_size=(boxes_act_reshaped.shape[1], boxes_act_reshaped.shape[2]), depth_multiplier=self.num_patterns,
                         strides=(1, 1), padding='same', activation=None, use_bias=False,
                         depthwise_initializer=w_init, pointwise_initializer='ones', trainable=False)
    self.conv2d = MySeparableConv2D(**conv2d_kwargs)

    # Embedding
    # See: https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding
    w_init = keras.initializers.Constant(boxes_qual_reshaped[self.zone])
    num_embedding_input_dim = (2 ** self.image_format[0])
    embedding_kwargs = dict(input_dim=num_embedding_input_dim, output_dim=1, input_length=1,
                            embeddings_initializer=w_init, trainable=False)
    self.embedding = k_layers.Embedding(**embedding_kwargs)

    # Dot product coeffs for packing the last axis
    self.po2_coeffs = (2 ** np.arange(self.image_format[0])).astype(np.int32)  # [1,2,4,8,16,32,64,128]

  def call(self, inputs):
    # Conv
    x = inputs  # NHWC, which is (None, 8, 288, 1)
    x = tf.cast(x, dtype=tf.float32)
    x = tf.transpose(x, perm=(0, 3, 2, 1))  # NHWC -> NCWH
    x = self.conv2d(x)  # NCWH -> NCWH', H' is dim of size H * D, D is depth_multipler
    x = tf.reshape(x, [-1, self.image_format[2], self.image_format[1], self.image_format[0], self.num_patterns])  # NCWH' -> NCWHD
    x = tf.transpose(x, perm=(0, 1, 2, 4, 3))  # NCWHD -> NCWDH
    x = tf.reduce_sum(x, axis=1)  # NCWDH -> NWDH, C is dim of size 1 and has been dropped

    # Pack 8 bits into a single number
    x = tf.clip_by_value(x, 0, 1)
    x = tf.cast(x, dtype=tf.int32)
    x = tf.reduce_sum(x * self.po2_coeffs, axis=-1)  # NWDH -> NWD, H has been packed into a single number and dropped
    x = tf.cast(x, dtype=tf.float32)

    # Embedding
    x = self.embedding(x)  # NWD -> NWDE, E is embedding output dim
    x = tf.reduce_sum(x, axis=-1)  # NWDE -> NWD, E is dim of size 1 and has been dropped
    x = tf.cast(x, dtype=tf.int32)

    # Gather max element
    idx_h = tf.argmax(x, axis=-1, output_type=tf.int32)  # NWD -> NW
    x = tf.gather(x, idx_h, axis=-1, batch_dims=2)  # NWD -> NW
    return (x, idx_h)

class Suppression(k_layers.Layer):
  def __init__(self, zone, image_format=image_format, **kwargs):
    super(Suppression, self).__init__(**kwargs)
    self.zone = zone
    self.image_format = image_format

  def call(self, inputs):
    x, idx_h = inputs

    # Non-max suppression
    x_padded = tf.pad(x, paddings=((0, 0), (1, 1)))  # ((pad_t, pad_b), (pad_l, pad_r))
    mask = (x > x_padded[:, :-2]) & (x >= x_padded[:, 2:])  # x > x_left && x >= x_right
    mask = tf.cast(mask, dtype=x.dtype)
    x = x * mask
    return (x, idx_h)

class ZoneSorting(k_layers.Layer):
  def __init__(self, zone, num_tracks=num_emtf_tracks, **kwargs):
    super(ZoneSorting, self).__init__(**kwargs)
    self.zone = zone
    self.num_tracks = num_tracks

  def call(self, inputs):
    x, idx_h = inputs

    # Sort (descending)
    idx_w = tf.argsort(x, axis=-1, direction='DESCENDING', stable=True)
    idx_w.set_shape(x.shape)
    idx_w = tf.transpose(tf.transpose(idx_w)[:self.num_tracks])  # truncate

    # Gather max elements
    x = tf.gather(x, idx_w, axis=-1, batch_dims=1)  # NW -> NW', W' is dim of size num_tracks
    idx_h = tf.gather(idx_h, idx_w, axis=-1, batch_dims=1)  # NW -> NW'
    return (x, idx_h, idx_w)

class ZoneMerging(k_layers.Layer):
  def __init__(self, num_tracks=num_emtf_tracks, **kwargs):
    super(ZoneMerging, self).__init__(**kwargs)
    self.num_tracks = num_tracks

  def call(self, inputs):
    x, idx_h, idx_w = inputs

    # Sort (descending)
    idx_z = tf.argsort(x, axis=-1, direction='DESCENDING', stable=True)
    idx_z.set_shape(x.shape)
    idx_z = tf.transpose(tf.transpose(idx_z)[:self.num_tracks])  # truncate

    # Gather max elements
    x = tf.gather(x, idx_z, axis=-1, batch_dims=1)  # NW' -> NW", W" is dim of size num_tracks
    idx_h = tf.gather(idx_h, idx_z, axis=-1, batch_dims=1)  # NW' -> NW"
    idx_w = tf.gather(idx_w, idx_z, axis=-1, batch_dims=1)  # NW' -> NW"
    idx_z = idx_z // self.num_tracks
    return (x, idx_h, idx_w, idx_z)

class TrkBuilding(k_layers.Layer):
  def __init__(self, num_features=num_emtf_features, num_tracks=num_emtf_tracks, **kwargs):
    super(TrkBuilding, self).__init__(**kwargs)
    self.num_features = num_features
    self.num_tracks = num_tracks

    # Set up py_func
    kwargs = dict(num_features=self.num_features)
    _build_track_cand = functools.partial(build_track_cand, **kwargs)
    #self.py_func = lambda x: tf.py_function(_build_track_cand, x, (tf.int32, tf.int32))
    self.py_func = lambda x: tf.numpy_function(_build_track_cand, x, (tf.int32, tf.int32))
    self.py_func_1 = lambda x: tf.map_fn(self.py_func, x, fn_output_signature=[tf.int32, tf.int32])
    self.py_func_2 = lambda x: tf.map_fn(self.py_func_1, x, fn_output_signature=[tf.int32, tf.int32])

  def call(self, inputs):
    x_ievt, x, idx_h, idx_w, idx_z = inputs

    # Call py_func
    x_ievt = tf.stack([x_ievt for _ in range(self.num_tracks)], axis=1)
    x = (x_ievt, x, idx_h, idx_w, idx_z)
    x = self.py_func_2(x)
    output_shape = (None,) + (self.num_tracks, self.num_features)
    x[0].set_shape(output_shape)
    output_shape = (None,) + (self.num_tracks, num_emtf_sites)
    x[1].set_shape(output_shape)
    return x

class DupeRemoval(k_layers.Layer):
  def __init__(self, num_features=num_emtf_features, num_tracks=num_emtf_tracks, **kwargs):
    super(DupeRemoval, self).__init__(**kwargs)
    self.num_features = num_features
    self.num_tracks = num_tracks

    # Set up py_func
    kwargs = dict(num_features=self.num_features, num_tracks=self.num_tracks)
    _remove_track_dupes = functools.partial(remove_track_dupes, **kwargs)
    #self.py_func = lambda x: tf.py_function(_remove_track_dupes, x, tf.int32)
    self.py_func = lambda x: tf.numpy_function(_remove_track_dupes, x, tf.int32)
    self.py_func_1 = lambda x: tf.map_fn(self.py_func, x, fn_output_signature=tf.int32)

  def call(self, inputs):
    x = inputs

    # Call py_func
    x = self.py_func_1(x)
    output_shape = (None,) + (self.num_tracks, self.num_features)
    x.set_shape(output_shape)
    return x

In [17]:
def create_model():
  # Input
  inputs = keras.Input(shape=(num_emtf_chambers, num_emtf_segments, num_emtf_variables), dtype='int32', name='inputs')
  x = inputs

  # Loop over zones
  x_list = []

  for i in range(num_emtf_zones):
    # Make zone images
    x_i = Zoning(zone=i, timezone=timezone, name='zoning_{0}'.format(i))(x)

    # Pattern recognition
    x_i = Pooling(zone=i, name='pooling_{0}'.format(i))(x_i)
    x_i = Suppression(zone=i, name='suppression_{0}'.format(i))(x_i)

    # Sort zone outputs
    x_i = ZoneSorting(zone=i, name='zonesorting_{0}'.format(i))(x_i)

    # Add x_i to x_list
    x_list.append(x_i)

  # Merge zone outputs
  i = next(iter(range(num_emtf_zones)))
  x = (k_layers.Concatenate(axis=-1)([x[0] for x in x_list]),
       k_layers.Concatenate(axis=-1)([x[1] for x in x_list]),
       k_layers.Concatenate(axis=-1)([x[2] for x in x_list]))
  x = ZoneMerging(name='zonemerging_{0}'.format(i))(x)

  # Track builder
  x = (inputs,) + x
  x = TrkBuilding(name='trkbuilding_{0}'.format(i))(x)
  x = DupeRemoval(name='duperemoval_{0}'.format(i))(x)

  # Output
  outputs = x

  # Model
  model = keras.Model(inputs=inputs, outputs=outputs, name='awesome_model')
  model.compile(optimizer="rmsprop", loss="sparse_categorical_crossentropy")

  # Summary
  model.summary()
  return model

model = create_model()

Model: "awesome_model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
inputs (InputLayer)             [(None, 115, 2, 13)] 0                                            
__________________________________________________________________________________________________
zoning_0 (Zoning)               (None, 8, 288, 1)    0           inputs[0][0]                     
__________________________________________________________________________________________________
zoning_1 (Zoning)               (None, 8, 288, 1)    0           inputs[0][0]                     
__________________________________________________________________________________________________
zoning_2 (Zoning)               (None, 8, 288, 1)    0           inputs[0][0]                     
______________________________________________________________________________________

### Evaluate model

In [18]:
class DataGenerator(keras.utils.Sequence):
  def __init__(self, x, batch_size=None, steps=None):
    self.x = x
    self.num_samples = int(x.shape[0])
    if not batch_size:
      batch_size = int(np.ceil(self.num_samples / float(steps))) if steps else 32
    self.batch_size = batch_size
    self.num_batches = int(np.ceil(self.num_samples / float(batch_size)))

  def __len__(self):
    return self.num_batches

  def __getitem__(self, index):
    #print('Processing batch {0}'.format(index))
    start, stop = (index * self.batch_size, min(self.num_samples, (index + 1) * self.batch_size))
    return self.x[start:stop]

In [19]:
# Make predictions

batch_size = 10000
outputs = model.predict(DataGenerator(inputs, batch_size=batch_size), workers=workers, use_multiprocessing=False)  # now wait...
logger.info('outputs: {0} type: {1}'.format(outputs.shape, type(outputs)))

[INFO    ] outputs: (652591, 4, 40) type: <class 'numpy.ndarray'>


In [20]:
# Write to file
if maxevents == -1:
  outfile = 'features_zone{0}.h5'.format(zone)
  outdict = {'features': da.from_array(outputs[:, 0, :]), 'truths': zone_part_l}
  da.to_hdf5(outfile, outdict, compression='lzf')
  logger.info('Wrote to {0}'.format(outfile))

[INFO    ] Wrote to features_zone0.h5


In [21]:
# Debug
tiny_inputs = inputs[:10]
model_zoning_0 = keras.Model(inputs=model.input, outputs=model.get_layer('zoning_0').output)
model_pooling_0 = keras.Model(inputs=model.input, outputs=model.get_layer('pooling_0').output)
model_suppression_0 = keras.Model(inputs=model.input, outputs=model.get_layer('suppression_0').output)
model_zonesorting_0 = keras.Model(inputs=model.input, outputs=model.get_layer('zonesorting_0').output)
model_zonemerging_0 = keras.Model(inputs=model.input, outputs=model.get_layer('zonemerging_0').output)
model_trkbuilding_0 = keras.Model(inputs=model.input, outputs=model.get_layer('trkbuilding_0').output)
model_duperemoval_0 = keras.Model(inputs=model.input, outputs=model.get_layer('duperemoval_0').output)

outputs = model_zoning_0(tiny_inputs)
x = tf_constant_value(outputs[0])
with np.printoptions(linewidth=100, threshold=1000):
  print('zoning_0_out:')
  print(x[0].nonzero())
  print(x[2].nonzero())
  print(x[5].nonzero())

outputs = model_pooling_0(tiny_inputs)
x = tf_constant_value(outputs[0])
with np.printoptions(linewidth=100, threshold=1000):
  print('pooling_0_out:')
  print(x[0].nonzero())
  print(x[2].nonzero())
  print(x[5].nonzero())

outputs = model_suppression_0(tiny_inputs)
x = tf_constant_value(outputs[0])
with np.printoptions(linewidth=100, threshold=1000):
  print('suppression_0_out:')
  print(x[0].nonzero(), x[0][x[0].nonzero()])
  print(x[2].nonzero(), x[2][x[2].nonzero()])
  print(x[5].nonzero(), x[5][x[5].nonzero()])

outputs = model_zonesorting_0(tiny_inputs)
x = tf_constant_value(outputs[0])
with np.printoptions(linewidth=100, threshold=1000):
  print('zonesorting_0_out:')
  print(x[0].nonzero(), x[0][x[0].nonzero()])
  print(x[2].nonzero(), x[2][x[2].nonzero()])
  print(x[5].nonzero(), x[5][x[5].nonzero()])

outputs = model_zonemerging_0(tiny_inputs)
x = tf_constant_value(outputs[0])
with np.printoptions(linewidth=100, threshold=1000):
  print('zonemerging_0_out:')
  print(x[0].nonzero(), x[0][x[0].nonzero()])
  print(x[2].nonzero(), x[2][x[2].nonzero()])
  print(x[5].nonzero(), x[5][x[5].nonzero()])

outputs = model_trkbuilding_0(tiny_inputs)
x = tf_constant_value(outputs[0])
x[x == ma_fill_value] = 0
with np.printoptions(linewidth=100, threshold=1000):
  print('trkbuilding_0_out:')
  print(x[0])
  print(x[2])
  print(x[5])

outputs = model_duperemoval_0(tiny_inputs)
x = tf_constant_value(outputs)
x[x == ma_fill_value] = 0
with np.printoptions(linewidth=100, threshold=1000):
  print('duperemoval_0_out:')
  print(x[0])
  print(x[2])
  print(x[5])

zoning_0_out:
(array([127]), array([0]))
(array([132]), array([0]))
(array([143, 149]), array([0, 0]))
pooling_0_out:
(array([ 93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110,
       111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128,
       129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146,
       147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161]),)
(array([230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247,
       248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265,
       266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283,
       284, 285, 286, 287]),)
(array([148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165,
       166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181